In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
class SDFDataset(Dataset):
    """Dataset for loading SDF csv file."""
    
    def __init__(self, csv_filename, root_dir='.'):
        """
        Args:
            csv_file (string): path to the csv file.
            root_dir (string): directory with data."""
        
        self.full_path = os.path.join(root_dir, csv_filename)
        self.df = pd.read_csv(self.full_path).to_numpy()
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        return self.df[idx]
    
    
class SDFDatasetFromDF(Dataset):
    """Dataset for loading SDF dataframe."""
    
    def __init__(self, df):
        """
        Args:
            df (pd.DataFrame): pandas dataframe with data."""
        
        self.df = df.to_numpy()
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        return self.df[idx]

In [3]:
# Checking SDFDataset
folder = '02818832'
model_dir = '10c15151ebe3d237240ea0cdca7b391a'
root_dir = os.path.join('data', folder, model_dir, 'models')

dataset = SDFDataset('sdf.csv', root_dir=root_dir)
dataset[:5]

array([[ 0.06561273,  0.3669782 , -0.39920893,  0.00078972],
       [ 0.34411201,  0.38627106,  0.17548348, -0.00232207],
       [-0.08663495,  0.30739027, -0.38965395,  0.00155663],
       [-0.20267186,  0.45919091, -0.17262226,  0.00096568],
       [-0.05268847,  0.22936745,  0.48423633, -0.00202632]])

In [4]:
dataloader = DataLoader(dataset, batch_size=32,
                        shuffle=True, num_workers=4)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched.size())
    if i_batch == 4:
        break

0 torch.Size([32, 4])
1 torch.Size([32, 4])
2 torch.Size([32, 4])
3 torch.Size([32, 4])
4 torch.Size([32, 4])


In [5]:
# Loading data in .csv format
df = pd.read_csv(os.path.join(root_dir, 'sdf.csv'))
df.describe()

,x,y,z,sdf
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.073887,-0.033628,-0.102642,0.013775
std,0.234475,0.353427,0.491546,0.068524
min,-0.983944,-0.988168,-0.995850,-0.109368
25%,-0.144064,-0.358193,-0.567773,-0.000533
50%,0.086747,-0.111310,-0.131580,0.000235
75%,0.311107,0.324987,0.353356,0.000913
max,0.991839,0.990133,0.980318,0.792862


In [6]:
# Checking SDFDatasetFromDF

# Splitting into training and testing data.
train_data, test_data = train_test_split(df)

train_dataset = SDFDatasetFromDF(train_data)
test_dataset = SDFDatasetFromDF(test_data)

train_dataset[:5]

array([[ 3.46513540e-01,  3.56236666e-01, -1.80856764e-01,
         6.10453426e-04],
       [ 1.17236823e-01,  4.04113799e-01, -5.70215046e-01,
        -1.93676760e-03],
       [ 1.04545362e-01, -2.07269251e-01, -7.77650714e-01,
        -8.69859825e-04],
       [ 1.43708557e-01,  4.06823397e-01, -5.73988914e-01,
        -1.20211858e-03],
       [ 1.09590739e-01, -2.20363244e-01,  7.64812052e-01,
         4.87884885e-04]])

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=32,
                              shuffle=True, num_workers=4)

for i_batch, sample_batched in enumerate(train_dataloader):
    print(i_batch, sample_batched.size())
    if i_batch == 4:
        break

0 torch.Size([32, 4])
1 torch.Size([32, 4])
2 torch.Size([32, 4])
3 torch.Size([32, 4])
4 torch.Size([32, 4])


In [18]:
class SimpleDeepSDF(nn.Module):
    """
    DeepSDF for one 3d shape.
    """
    
    def __init__(self):
        super(SimpleDeepSDF, self).__init__()
        
        # input of 3 - (x,y,z)
        self.fc1 = nn.Linear(3, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 509)
        # between this layers we use input again
        self.fc5 = nn.Linear(512, 512)
        self.fc6 = nn.Linear(512, 512)
        self.fc7 = nn.Linear(512, 512)
        self.fc8 = nn.Linear(512, 1)
        self.tanh = nn.Tanh()  # activation layer

    def forward(self, x):
        
        inpt = x.detach().clone().type(torch.FloatTensor)  # input vector

        x = F.relu(self.fc1(inpt))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        # using input again
        x = torch.cat((x, inpt), dim=1)
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)

        x = torch.tanh(x)  # activation

        return x

In [14]:
class DeepSDF(nn.Module):
    """
    DeepSDF for multiple 3d shapes.
    """
    
    def __init__(self):
        super(DeepSDF, self).__init__()
        
        # Latent vector needs to be reinitialized
        self.lat_vec = np.random.sample(256)
        
        # input of 259 -> 256 - latent vector, 3 - (x,y,z)
        self.fc1 = nn.Linear(259, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 253)
        # between this layers we use input again
        self.fc5 = nn.Linear(512, 512)
        self.fc6 = nn.Linear(512, 512)
        self.fc7 = nn.Linear(512, 512)
        self.fc8 = nn.Linear(512, 1)

    def forward(self, x):
        
        inpt = np.concatenate(self.lat_vec, x)  # input vector

        x = F.relu(self.fc1(inpt))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        # using input again
        x = np.concatenate(x, inpt)
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = self.tanh(x)  # activation
        
        return x
    
    # TODO
    def load_latent_vector(filename):
        pass
    
    # TODO
    def save_latent_vector(filename):
        pass

In [19]:
net = SimpleDeepSDF()
print(net)

SimpleDeepSDF(
  (fc1): Linear(in_features=3, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=509, bias=True)
  (fc5): Linear(in_features=512, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=512, bias=True)
  (fc7): Linear(in_features=512, out_features=512, bias=True)
  (fc8): Linear(in_features=512, out_features=1, bias=True)
  (tanh): Tanh()
)


In [20]:
def sdf_loss(output, target, dlt=0.5):
    return torch.sum(torch.abs(torch.clamp(output, -dlt, dlt) - torch.clamp(target, -dlt, dlt).view(-1, 1)))
    
criterion = sdf_loss  # Loss
optimizer = optim.Adam(net.parameters())

In [21]:
# training
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[:, :3], data[:, 3]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

[1,  1000] loss: 0.370
[1,  2000] loss: 0.309
[2,  1000] loss: 0.304
[2,  2000] loss: 0.270
[3,  1000] loss: 0.257
[3,  2000] loss: 0.222
[4,  1000] loss: 0.229
[4,  2000] loss: 0.203
[5,  1000] loss: 0.130
[5,  2000] loss: 0.118
[6,  1000] loss: 0.114
[6,  2000] loss: 0.118
[7,  1000] loss: 0.109
[7,  2000] loss: 0.114
[8,  1000] loss: 0.109
[8,  2000] loss: 0.110
[9,  1000] loss: 0.105
[9,  2000] loss: 0.104
[10,  1000] loss: 0.103
[10,  2000] loss: 0.101
[11,  1000] loss: 0.106
[11,  2000] loss: 0.098
[12,  1000] loss: 0.100
[12,  2000] loss: 0.103
[13,  1000] loss: 0.102
[13,  2000] loss: 0.101
[14,  1000] loss: 0.101
[14,  2000] loss: 0.095
[15,  1000] loss: 0.094
[15,  2000] loss: 0.100
[16,  1000] loss: 0.094
[16,  2000] loss: 0.096
[17,  1000] loss: 0.097
[17,  2000] loss: 0.091
[18,  1000] loss: 0.090
[18,  2000] loss: 0.097
[19,  1000] loss: 0.095
[19,  2000] loss: 0.091
[20,  1000] loss: 0.089
[20,  2000] loss: 0.092
Finished Training


In [22]:
torch.save(net, os.path.join('models', 'third_model'))

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SimpleDeepSDF. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Loading model + Testing

In [23]:
net = torch.load(os.path.join('models', 'third_model'))
net

SimpleDeepSDF(
  (fc1): Linear(in_features=3, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=509, bias=True)
  (fc5): Linear(in_features=512, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=512, bias=True)
  (fc7): Linear(in_features=512, out_features=512, bias=True)
  (fc8): Linear(in_features=512, out_features=1, bias=True)
  (tanh): Tanh()
)

In [24]:
# Testing and saving results for graphic representation
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=0)

text = 'x,y,z,sdf\n'
loss = 0
with torch.no_grad():
    for data in test_dataloader:
        input, labels = data[:, :3], data[:, 3]
        
        outputs = net(input)
        text += '{},{},{},{}\n'.format(input[0][0], input[0][1], input[0][2], outputs[0][0])
        loss += (np.abs(outputs - labels)).sum()

print('Average loss of all test data: %f' % (loss / len(test_dataset)))

with open(os.path.join('data', folder, model_dir, 'models', 'result.csv'), 'w') as f:
    f.write(text)
    
print('"result.csv" is generated.')

Average loss of all test data: 0.002843
"result.csv" is generated.


In [25]:
output = net(data[:, :3])
print("Model outputs [{}], while the true value is [{}].".format(output[0, 0], data[0, 3]))

Model outputs [0.0005956334643997252], while the true value is [-0.0019950037822127342].
